In [10]:
from anomaly_detection import config, dataset, features
import numpy as np

# set seed for reproducibility
np.random.seed(42)

In [3]:
file_path = '/Users/emadchelhi/Desktop/git_repos/Why-anomalous/data/raw/Wholesale customers data.csv'

# load the data
tab_df = dataset.load_data(file_path)

Data successfully loaded from /Users/emadchelhi/Desktop/git_repos/Why-anomalous/data/raw/Wholesale customers data.csv.


In [4]:
# excluding the categorical columns
del tab_df['Channel']
del tab_df['Region']

In [11]:
# log-transform the data
tab_df = features.logarithm(tab_df, 100)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().